# Sentinel metadata generator

    Author: Ben Ross
    Date: 27th April 2021
    
Purpose: 

In [3]:
import folium
# from sentinelsat.sentinel import SentinelAPI, read_geojson, geojson_to_wkt
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt
from datetime import date
import pandas as pd
import geopandas as gpd

## Define the search geometries

In [130]:
# brisbane = geojson_to_wkt(read_geojson(r'files\brisbane.geojson'))
mines = gpd.read_file(r'C:\Users\rossb1\OneDrive - ITP (Queensland Government)\gis\playground\playground.gdb', layer='coal_mines_map_series_shapes')
mines.to_crs('epsg:4326', inplace=True)

C:\Anaconda\envs\py37\lib\site-packages\geopandas\geodataframe.py:577: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


In [131]:
mines['Project_Name']

0               The Range
1            Wilkie Creek
2              New Acland
3    Wandoan Coal Project
4                Elimatta
5             Kogan Creek
6               Commodore
7            Cameby Downs
Name: Project_Name, dtype: object

In [169]:
wilkiecreek = mines.iloc[1]['geometry']
newacland = mines.iloc[2]['geometry']
kogancreek = mines.iloc[5]['geometry']
commodore = mines.iloc[6]['geometry']
cambeydowns = mines.iloc[7]['geometry']

In [4]:
model_domain = gpd.read_file(r'G:\OGIA\H_PoT\03_As_created\Spatial\01_Geology\geology.gdb', layer='regional_model_domain')
model_domain.to_crs('epsg:4326', inplace=True)

In [79]:
subsidence_model_domain = gpd.read_file(r'files/subsidence_model_extent.geojson')

## To obtain Sentinel-1 data

It seems like the api only returns 100 sentinel 1 products at a time. Thankfully the target area in the subsidence model domain only has 1 observation per day. Meaning we can easily target quarterly metadata queries. The syntax for the api is 'beginYYYYMMDD' and 'endYYYYMMDD'.

In [80]:
# footprint = model_domain.iloc[0]['geometry']
footprint = subsidence_model_domain.iloc[0]['geometry']

In [110]:
api = SentinelAPI('benross', 'password', 'https://scihub.copernicus.eu/dhus')

products = api.query(footprint, producttype='SLC', limit=100, date = ('20210101', '20210217'))

# products = api.query(footprint, date = ('20200901', date(2020, 12, 31)), producttype='SLC', orbitdirection='ASCENDING')

print('Products returned by search: {n}'.format(n=len(products)))
products_gdf = api.to_geodataframe(products)

Products returned by search: 48


In [111]:
products_gdf.to_file("files/products_S1AB_from01012021.geojson", driver='GeoJSON')

In [84]:
products_gdf.head()

,title,link,link_alternative,link_icon,summary,ondemand,beginposition,endposition,ingestiondate,missiondatatakeid,...,producttype,platformname,platformidentifier,instrumentname,instrumentshortname,productclass,polarisationmode,acquisitiontype,uuid,geometry
3e44bc65-ddf0-4938-9dea-00ea6c5581ba,S1A_IW_SLC__1SSH_20210602T083311_20210602T0833...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2021-06-02T08:33:11.24Z, Instrument: SAR...",false,2021-06-02 08:33:11.240,2021-06-02 08:33:39.128,2021-06-02 10:45:48.022,295145,...,SLC,Sentinel-1,2014-016A,Synthetic Aperture Radar (C-band),SAR-C SAR,S,HH,NOMINAL,3e44bc65-ddf0-4938-9dea-00ea6c5581ba,"MULTIPOLYGON (((150.07457 -27.82059, 152.55756..."
64a97da4-2c99-4849-a687-479346c0cd4c,S1A_IW_SLC__1SSH_20210602T083243_20210602T0833...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2021-06-02T08:32:43.652Z, Instrument: SA...",false,2021-06-02 08:32:43.652,2021-06-02 08:33:13.472,2021-06-02 10:39:52.379,295145,...,SLC,Sentinel-1,2014-016A,Synthetic Aperture Radar (C-band),SAR-C SAR,S,HH,NOMINAL,64a97da4-2c99-4849-a687-479346c0cd4c,"MULTIPOLYGON (((150.54662 -29.47242, 153.06992..."
a4d492ac-04bc-44b6-b7cb-7e3d76073de8,S1A_IW_SLC__1SDV_20210528T192144_20210528T1922...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2021-05-28T19:21:44.988Z, Instrument: SA...",false,2021-05-28 19:21:44.988,2021-05-28 19:22:11.943,2021-05-28 22:10:08.900,294625,...,SLC,Sentinel-1,2014-016A,Synthetic Aperture Radar (C-band),SAR-C SAR,S,VV VH,NOMINAL,a4d492ac-04bc-44b6-b7cb-7e3d76073de8,"MULTIPOLYGON (((151.91672 -26.80047, 152.35709..."
993b7548-109d-46da-95b8-f9e44a61ffc3,S1A_IW_SLC__1SDV_20210528T192209_20210528T1922...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2021-05-28T19:22:09.811Z, Instrument: SA...",false,2021-05-28 19:22:09.811,2021-05-28 19:22:36.770,2021-05-28 21:52:01.362,294625,...,SLC,Sentinel-1,2014-016A,Synthetic Aperture Radar (C-band),SAR-C SAR,S,VV VH,NOMINAL,993b7548-109d-46da-95b8-f9e44a61ffc3,"MULTIPOLYGON (((151.50015 -28.28812, 151.95167..."
ed8aba0c-3e15-437a-906c-830bc6e5630e,S1A_WV_SLC__1SSV_20210528T082016_20210528T0828...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,"Date: 2021-05-28T08:20:16.887Z, Instrument: SA...",false,2021-05-28 08:20:16.887,2021-05-28 08:28:52.487,2021-05-28 11:55:19.149,294572,...,SLC,Sentinel-1,2014-016A,Synthetic Aperture Radar (C-band),SAR-C SAR,S,VV,NOMINAL,ed8aba0c-3e15-437a-906c-830bc6e5630e,"MULTIPOLYGON (((156.76027 -44.73029, 157.02048..."


## To obtain Sentinel 2 data

In [178]:
# by default sentinelsat qill return results from the last 24 hours only

api = SentinelAPI('benross', 'password', 'https://scihub.copernicus.eu/apihub')

products = api.query(footprint, date = ('20180601', '20180630'), platformname = 'Sentinel-2', cloudcoverpercentage = (0, 30))

print('Products returned by search: {n}'.format(n=len(products)))
products_gdf = api.to_geodataframe(products)

Products returned by search: 19


C:\Anaconda\envs\py37\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [32]:
products_gdf.columns

# products_gdf['cloudcoverpercentage']

Index(['title', 'link', 'link_alternative', 'link_icon', 'summary', 'ondemand',
       'beginposition', 'endposition', 'ingestiondate', 'missiondatatakeid',
       'orbitnumber', 'lastorbitnumber', 'relativeorbitnumber',
       'lastrelativeorbitnumber', 'filename', 'format', 'identifier', 'size',
       'status', 'timeliness', 'sensoroperationalmode', 'swathidentifier',
       'orbitdirection', 'producttype', 'platformname', 'platformidentifier',
       'instrumentname', 'instrumentshortname', 'productclass',
       'polarisationmode', 'acquisitiontype', 'uuid', 'slicenumber',
       'geometry'],
      dtype='object')

In [181]:
products_gdf.to_file("files/products_S2AB.geojson", driver='GeoJSON')

## Display the search results from either satellite on a folium map

In [93]:
m = folium.Map([-27.00, 151.50], zoom_start=7)
folium.GeoJson(subsidence_model_domain).add_to(m)
folium.GeoJson(products_gdf['geometry']).add_to(m)
# folium.GeoJson(model_domain['geometry']).add_to(m) # simply pick an index from the list above to change the target mine

m

In [34]:
m = folium.Map([-27.00, 151.00], zoom_start=8)
folium.GeoJson(products_gdf.iloc[4]['geometry']).add_to(m)
folium.GeoJson(mines.iloc[1]['geometry']).add_to(m) # simply pick an index from the list above to change the target mine
m

NameError: name 'mines' is not defined